# IPython/Jupyter Widgets

Matthias Bussonnier

BussonnierMatthias@gmail.com

### Disclamer

This is mainly the work on Jonathan Frederic (CalPoly), Jason Grout and Sylvain Corlay (Bloomberg)

## What are widgets ?

The goal of Jupyter and IPython project is to minimize the amount of effor to explore and communicate results about data analysis.

In exploratory data analysis, we often need to tweek parameters. Widgets are here to improve
the ability to tweek and explore parameter both from user and developper point of view.
Widgets allow quick manipulation, and developement of interactive functions. 

One of the goals of the Jupyter Notebook is to minimize the “distance” the user is from their data.  This means allowing the user to quickly view and manipulate the data.  

| ![](images/inputoutput.PNG)                                                                           | ![](images/widgets.PNG)                                                                                                                      |
|-------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------|
| Before the widgets, this was just the segmentation of code and results from executing those segments. | Widgets further decrease the distance between the user and their data by allowing UI interactions to directly manipulate data in the kernel. |

## Disclamer (again)

What I'll show here is the Python-specific implementation, the same (with different API) is availlable in other languages. Other languages might, or might not be better adapted to have widgets.

# Interact

The `interact` function (`IPython.html.widgets.interact`) automatically creates user interface (UI) controls for exploring code and data interactively. It is the easiest way to get started using IPython's widgets.

In [ ]:
from __future__ import print_function
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets

## Basic `interact`

At the most basic level, `interact` autogenerates UI controls for function arguments, and then calls the function with those arguments when you manipulate the controls interactively. To use `interact`, you need to define a function that you want to explore. Here is a function that prints its only argument `x`.

In [ ]:
def f(x):
    print(x)


When you pass this function as the first argument to `interact` along with an integer keyword argument (`x=10`), a slider is generated and bound to the function.

In [ ]:
interact(f, x=10);

When you move the slider, the function is called and the current value of `x` is printed.

If you pass `True` or `False`, `interact` will generate a checkbox:

In [ ]:
interact(f, x=True)

If you pass a string, `interact` will generate a text area.

In [ ]:
interact(f, x='Hi there!');

`interact` can also be used as a decorator. This allows you to define a function and interact with it in a single

In [ ]:
@interact
def g(x:True, y=1.0, z=fixed('not interactive')):
    print(x, y, z)

## Widget abbreviations

In [ ]:
@interact(y=(0.1,10.,2.))
def g(x:True, y=1.0, z=fixed('- not interactive')):
    print(x, y, z)

When you pass an integer valued keyword argument (`x=10`) to `interact`, it generates an integer valued slider control with a range of $[-10,+3\times10]$. In this case `10` is an *abbreviation* for an actual slider widget:

```python
IntSlider(min=-10,max=30,step=1,value=10)
```

In fact, we can get the same result if we pass this `IntSlider` as the keyword argument for `x`:

In [ ]:
interact(f, x=widgets.IntSlider(min=-10, max=30, step=1, value=10));

In [ ]:
interact(f, x=(-10, 30, 1));

In [ ]:
interact(f, x=10)

This examples clarifies how `interact` proceses its keyword arguments:

1. If the keyword argument is `Widget` instance with a `value` attribute, that widget is used. Any widget with a `value` attribute can be used, even custom ones.
2. Otherwise, the value is treated as a *widget abbreviation* that is converted to a widget before it is used.

The following table gives an overview of different widget abbreviations:

<table class="table table-condensed table-bordered">
  <tr><td><strong>Keyword argument</strong></td><td><strong>Widget</strong></td></tr>  
  <tr><td>`True` or `False`</td><td>Checkbox</td></tr>  
  <tr><td>`'Hi there'`</td><td>Text</td></tr>
  <tr><td>`value` or `(min,max)` or `(min,max,step)` if integers are passed</td><td>IntSlider</td></tr>
  <tr><td>`value` or `(min,max)` or `(min,max,step)` if floats are passed</td><td>FloatSlider</td></tr>
  <tr><td>`('orange','apple')` or `{'one':1,'two':2}`</td><td>Dropdown</td></tr>
</table>

In [ ]:
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import skimage.data as data


In [ ]:
@interact(cmap=dir(matplotlib.pylab.cm),
          interpolation=matplotlib.image._AxesImageBase.interpnames,
         img=('camera', 'checkerboard','clock', 'coins', 'horse', 'moon', 'text'))
def pl(cmap='gray', interpolation='None', img='camera', n=15):
    r = getattr(data, img)()[::n,::n]
    plt.imshow(r, cmap=cmap, interpolation=interpolation)

# Interactive

In addition to `interact` IPython provides another function, `interactive`, that is useful when you want to reuse the widget that are produced or access the data that is bound to the UI controls.

Here is a function that returns the sum of its two arguments.

In [ ]:
def f(a, b):
    return a+b

Unlike `interact`, `interactive` returns a `Widget` instance rather than immediately displaying the widget.

In [ ]:
w = interactive(f, a=10, b=20)

In [ ]:
from IPython.display import display
display(w)

At this point, the UI controls work just like they would if `interact` had been used. You can manipulate them interactively and the function will be called. However, the widget instance returned by `interactive` also give you access to the current keyword arguments and return value of the underlying Python function.

Here are the current keyword arguments. If you rerun this cell after manipulating the sliders, the values will have changed.

In [ ]:
w.children[0].value = 17

In [ ]:
w.kwargs

In [ ]:
w.children[0]

In [ ]:
w.children[1]

In [ ]:
from IPython.html.widgets import IntSlider, jslink
wmax = IntSlider(value=20, min=10, max=20)

jslink((wmax,'value'),(w.children[0], 'max'))

wmax

### Complex UI

In [ ]:
name = widgets.Text(description='Color:', padding=4)
color = widgets.Dropdown(description='Background Color:', padding=4, options=['white','darkred', 'orange', 'deepblue', 'green', 'blue', 'indigo', 'violet'])
page1 = widgets.Box(children=[name, color], padding=4)

age = widgets.IntSlider(description='Age:', padding=4, min=0, max=120, value=50)
gender = widgets.RadioButtons(description='LBL Staff:', padding=4, options=['yes', 'no', 'other'])
page2 = widgets.Box(children=[age, gender], padding=4)

tabs = widgets.Tab(children=[page1, page2])
display(tabs)

widgets.jslink((color, "selected_label") , (name, 'background_color'))
widgets.jslink((name, "value") , (name, 'color'))
widgets.jslink((name, "value") , (color, 'color'))
widgets.jslink((color, "selected_label") , (name, 'background_color'))
widgets.jslink((color, "selected_label") , (color, 'background_color'))

tabs.set_title(0, 'Name')
tabs.set_title(1, 'Details')

## full example

In [ ]:
%matplotlib inline
from IPython.html.widgets import interact, interactive
from IPython.display import clear_output, display, HTML
import numpy as np
from scipy import integrate

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import cnames
from matplotlib import animation

In [ ]:
def solve_lorenz(N=10, angle=0.0, max_time=4.0, σ=10.0, β=8./3, ρ=28.0):

    fig = plt.figure()
    ax = fig.add_axes([0, 0, 1, 1], projection='3d')
    ax.axis('off')

    # prepare the axes limits
    ax.set_xlim((-25, 25))
    ax.set_ylim((-35, 35))
    ax.set_zlim((5, 55))
    
    def lorenz_deriv(x_y_z, t0, σ=σ, β=β, ρ=ρ):
        """Compute the time-derivative of a Lorenz system."""
        x, y, z = x_y_z
        return [σ * (y - x), x * (ρ - z) - y, x * y - β * z]

    # Choose random starting points, uniformly distributed from -15 to 15
    np.random.seed(1)
    x0 = -15 + 30 * np.random.random((N, 3))

    # Solve for the trajectories
    t = np.linspace(0, max_time, int(250*max_time))
    x_t = np.asarray([integrate.odeint(lorenz_deriv, x0i, t)
                      for x0i in x0])
    
    # choose a different color for each trajectory
    colors = plt.cm.jet(np.linspace(0, 1, N))

    for i in range(N):
        x, y, z = x_t[i,:,:].T
        lines = ax.plot(x, y, z, '-', c=colors[i])
        plt.setp(lines, linewidth=2)

    ax.view_init(30, angle)
    plt.show()

    return t, x_t

In [ ]:
ww = interactive(matplotlib.style.use, name=list(matplotlib.style.library.keys()))
w = interactive(solve_lorenz, angle=(0.,360.), σ=(0.0,50.0), ρ=(0.0,50.0), β=fixed(3))
display(ww, w)

# Custom widgets

Widgets

In [ ]:
from IPython.html.widgets import DOMWidget
from IPython.display import display
from IPython.utils.traitlets import Unicode, Bool
class ColorWidget(DOMWidget):
    _view_module = Unicode('ColorViewModule', sync=True)
    _view_name = Unicode('ColorView', sync=True)
    
    ## define values that will be synced
    value = Unicode('#990000', sync=True)
    type = Unicode('color', sync=True)
    verbose = Bool(False, sync=True)
    
    
    def _value_changed(self, old, new):
        if(self.verbose):
            print('value have changed from ', old, 'to', new)

In [ ]:
%%javascript
delete requirejs.s.contexts._.defined.ColorViewModule;
define('ColorViewModule', ['jquery', 'widgets/js/widget'], function($, widget) {
    
    var ColorView = widget.DOMWidgetView.extend({
        render: function() {
            // create a dom element:
            this.colorpicker = $('<input/>');
            this.$el.append(this.colorpicker);
            
            // bind some backbone.js events to update the dom on change
            this.listenTo(this.model, 'change:value', this._update_value, this);
            this.listenTo(this.model, 'change:type', this._update_value, this);
            this._update_value();
            
            this.colorpicker.on('change', this._set_value.bind(this));
        },
        _update_value: function() {
            this.colorpicker.val(this.model.get('value'));
            this.colorpicker.attr('type', this.model.get('type'));
        },
        _set_value: function() {
            this.model.set('value', this.colorpicker.val());
            this.touch();
        }
    });
    
    return {ColorView: ColorView};
});


In [ ]:
w = ColorWidget()
display(w,w)

In [ ]:
w.value = '#44AAFF'

In [ ]:
w

In [ ]:
w.value

In [ ]:
w.verbose = True

In [ ]:
w.type = 'text'

## Part 2

In [ ]:
%%html
<link rel="stylesheet" type="text/css" href="../style.css">
<script>require(['notebook/js/scrollmanager'], function(sm) { IPython.notebook.scroll_manager = new sm.SlideScrollManager(IPython.notebook); });</script>